In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.0
pandas 0.25.3
sklearn 0.22
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
## 大纲
##  1、载入数据
##   1.1、数据处理，提取所有类别,并转化为类别词表
##   1.2、数据分词，并存储，为训练时加载使用
##   1.3、为构建词表使用，并存储词表  


In [3]:
dir_train='data\\cnews.train.txt'
dir_val='data\\cnews.val.txt'
dir_test='data\\cnews.test.txt'

# prepare data output files
seg_train_file = 'data\\cnews.train.seg.txt'
seg_val_file = 'data\\cnews.val.seg.txt'
seg_test_file = 'data\\cnews.test.seg.txt'

vocab_file = 'data\\cnews.vocab.txt'
label_vocab_file = 'data\\cnews.label.vocab.txt'

def load_data(path,num_lines):
    lines = open(path,encoding='UTF-8').read().strip().split('\n')
    example_pairs = [ [w for w in line.split('\t')] for line in lines[:num_lines]]
    return zip(*example_pairs)

def label_dict(y_val):
    index=1
    dd={}
    for label in y_val:
        if label not in dd.keys():
            dd[label]=index
            index +=1
    return dd

y_val,X_val= load_data(dir_val,None)
print('y_val length:',len(y_val))
print('X_val length:',len(X_val))

label_dict=label_dict(y_val)
print('label_dict:',label_dict)


y_val length: 5000
X_val length: 5000
label_dict: {'体育': 1, '娱乐': 2, '家居': 3, '房产': 4, '教育': 5, '时尚': 6, '时政': 7, '游戏': 8, '科技': 9, '财经': 10}


In [4]:
## 保存label文件内容
with open(label_vocab_file, 'w', encoding='utf-8') as f:
    f.seek(0) 
    f.truncate()   #清空文件
    for label,value in label_dict.items():
        line = '%s\t%d\n' % (label,value)
        print('%s\t%d' % (label,value))
        f.write(line)

体育	1
娱乐	2
家居	3
房产	4
教育	5
时尚	6
时政	7
游戏	8
科技	9
财经	10


In [5]:
## 做分词处理
## jieba分词
import jieba
def generate_seg_file(input_file, output_seg_file):
    """Segment the sentences in each line in input_file"""
    with open(input_file, 'r',encoding='utf-8') as f:
        lines = f.readlines()
    with open(output_seg_file, 'w',encoding='utf-8') as f:
        f.seek(0) 
        f.truncate()   #清空文件
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            word_iter = jieba.cut(content)
            word_content = ''
            for word in word_iter:
                word = word.strip(' ')
                if word != '':
                    word_content += word + ' '
            out_line = '%s\t%s\n' % (label, word_content.strip(' '))
            f.write(out_line)

## 时间比较长，只需执行一次
generate_seg_file(dir_train, seg_train_file)
print('完成train数据分词重写！')
generate_seg_file(dir_val, seg_val_file)
print('完成val数据分词重写！')
generate_seg_file(dir_test, seg_test_file)
print('完成test数据分词重写！')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jandy\AppData\Local\Temp\jieba.cache
Loading model cost 0.811 seconds.
Prefix dict has been built successfully.


完成train数据分词重写！
完成val数据分词重写！
完成test数据分词重写！


In [6]:
## 做词表,就是词出现的次数频率
def generate_vocab_file(input_seg_file, output_vocab_file):
    with open(input_seg_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    word_dict = {}
    for line in lines:
        label, content = line.strip('\r\n').split('\t')
        for word in content.split():
            word_dict.setdefault(word, 0)
            word_dict[word] += 1
    # [(word, frequency), ..., ()]
    ##  按词频排序
    sorted_word_dict = sorted(
        word_dict.items(), key = lambda d:d[1], reverse=True)
    with open(output_vocab_file, 'w',encoding='utf-8') as f:
        f.seek(0) 
        f.truncate()   #清空文件
        f.write('<UNK>\t10000000\n')
        for item in sorted_word_dict:
            f.write('%s\t%d\n' % (item[0], item[1]))
    print('dict length:',len(sorted_word_dict))

generate_vocab_file(seg_train_file, vocab_file)

dict length: 359260


In [7]:
## 另一种词表方案
vocab_file_simple = 'data\\cnews.vocab_simple.txt'
## 构建分词器
import tensorflow_datasets as tfds

with open(seg_train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    example_pairs=[[example for example in line.strip('\r\n').split('\t')] for line in lines]
    y,X_train_seg=zip(*example_pairs)
    
                

In [8]:
            
print(X_train_seg[:1])
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(X_train_seg, target_vocab_size= 2**16)
print('vocab_size:',tokenizer.vocab_size)
sub_words=tokenizer.subwords
print('sub_words length:',len(sub_words))

('马晓旭 意外 受伤 让 国奥 警惕 无奈 大雨 格外 青睐 殷家 军 记者 傅亚雨 沈阳 报道 来到 沈阳 ， 国奥队 依然 没有 摆脱 雨水 的 困扰 。 7 月 31 日 下午 6 点 ， 国奥队 的 日常 训练 再度 受到 大雨 的 干扰 ， 无奈 之下 队员 们 只 慢跑 了 25 分钟 就 草草收场 。 31 日 上午 10 点 ， 国奥队 在 奥体中心 外场 训练 的 时候 ， 天 就是 阴沉沉 的 ， 气象预报 显示 当天 下午 沈阳 就 有 大雨 ， 但 幸好 队伍 上午 的 训练 并 没有 受到 任何 干扰 。 下午 6 点 ， 当 球队 抵达 训练场 时 ， 大雨 已经 下 了 几个 小时 ， 而且 丝毫 没有 停下来 的 意思 。 抱 着 试一试 的 态度 ， 球队 开始 了 当天 下午 的 例行 训练 ， 25 分钟 过去 了 ， 天气 没有 任何 转好 的 迹象 ， 为了 保护 球员 们 ， 国奥队 决定 中止 当天 的 训练 ， 全队 立即 返回 酒店 。 在 雨 中 训练 对 足球队 来说 并 不是 什么 稀罕 事 ， 但 在 奥运会 即将 开始 之前 ， 全队 变得 “ 娇贵 ” 了 。 在 沈阳 最后 一周 的 训练 ， 国奥队 首先 要 保证 现有 的 球员 不再 出现意外 的 伤病 情况 以免 影响 正式 比赛 ， 因此 这一 阶段 控制 训练 受伤 、 控制 感冒 等 疾病 的 出现 被 队伍 放在 了 相当 重要 的 位置 。 而 抵达 沈阳 之后 ， 中 后卫 冯萧霆 就 一直 没有 训练 ， 冯萧霆 是 7 月 27 日 在 长春 患上 了 感冒 ， 因此 也 没有 参加 29 日 跟 塞尔维亚 的 热身赛 。 队伍 介绍 说 ， 冯萧霆 并 没有 出现 发烧 症状 ， 但 为了 安全 起 见 ， 这 两天 还是 让 他 静养 休息 ， 等 感冒 彻底 好 了 之后 再 恢复 训练 。 由于 有 了 冯萧霆 这个 例子 ， 因此 国奥队 对雨中 训练 就 显得 特别 谨慎 ， 主要 是 担心 球员 们 受凉 而 引发 感冒 ， 造成 非战斗 减员 。 而 女足 队员 马晓旭 在 热身赛 中 受伤 导致 无缘 奥运 的 前科 ， 也 让 在 沈阳 的 国奥队 现在 格外 警惕 ， “ 训练 中 不断 嘱咐 队员 们 要

In [9]:
for item in sub_words[:10]:
    print(item)
with open(vocab_file_simple, 'w',encoding='utf-8') as f:
    f.seek(0) 
    f.truncate()   #清空文件
    f.write('<UNK>\t10000000\n')
    for index,item in enumerate(sub_words):
        f.write('%s\t%d\n' % (item,index))
    print('dict length:',len(sub_words))

 ， 
的_
 。 
在_
 、 
是_
了_
和_
也_
 “ 
dict length: 64708
